In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from unidecode import unidecode 

In [2]:
df_movies = pd.read_csv("Movies_EDA.csv", sep=",")

In [3]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42195 entries, 0 to 42194
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               42195 non-null  int64  
 1   title            42195 non-null  object 
 2   overview         42195 non-null  object 
 3   release_date     42195 non-null  object 
 4   release_year     42195 non-null  int64  
 5   budget           42195 non-null  float64
 6   return           42195 non-null  float64
 7   popularity       42195 non-null  float64
 8   vote_average     42195 non-null  float64
 9   vote_count       42195 non-null  float64
 10  revenue          42195 non-null  float64
 11  runtime          42195 non-null  float64
 12  genres_names     42195 non-null  object 
 13  names_actors     42195 non-null  object 
 14  director         42195 non-null  object 
 15  companies_names  42195 non-null  object 
 16  name_collection  42195 non-null  object 
 17  countries_na

Solo tomaremos las columnas que necesitemos para el sistema de recomendación.

In [5]:
df_movies = df_movies[["title", "overview", "release_year", "genres_names", "runtime"]].copy()

In [6]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42195 entries, 0 to 42194
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         42195 non-null  object 
 1   overview      42195 non-null  object 
 2   release_year  42195 non-null  int64  
 3   genres_names  42195 non-null  object 
 4   runtime       42195 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.6+ MB


Eliminamos los duplicados de la columna title.

In [6]:
df_movies.drop_duplicates(subset=['title'],inplace=True)
df_movies=df_movies.reset_index(drop=True)
df_movies.shape

(42195, 5)

In [7]:
print(df_movies.head)

<bound method NDFrame.head of                              title  \
0                        Toy Story   
1                          Jumanji   
2                 Grumpier Old Men   
3                Waiting to Exhale   
4      Father of the Bride Part II   
...                            ...   
42190          The Burkittsville 7   
42191              Caged Heat 3000   
42192          Century of Birthing   
42193             Satan Triumphant   
42194                     Queerama   

                                                overview  release_year  \
0      Led by Woody, Andy's toys live happily in his ...          1995   
1      When siblings Judy and Peter discover an encha...          1995   
2      A family wedding reignites the ancient feud be...          1995   
3      Cheated on, mistreated and stepped on, the wom...          1995   
4      Just when George Banks has recovered from his ...          1995   
...                                                  ...           ..

        ► Sistema de recomendación.

Se realizará un sistema de recomendación de películas basados en el resumen y el género de la película de entrada. 

El resultado incluirá el título de la película, el año de estreno y la duración de la película recomendada.


Lo que se realizó en la función fue:

*Normalizamos el título ingresado por el usuario para que no distinga entre mayúsculas y minúsculas, y no tenga en cuenta los acentos.*

*Se obtiene la película de entrada por título en la base de datos.*
 
*Calculamos la similitud entre la película de entrada y las demás películas en función del resumen.*
    
*Se filtró las películas similares por género.*
    
*Se ordenó las películas similares según la similitud.*
    
*Conversión de los datos de duración de las recomendaciones de tipo float a int.*
    
*Retorno de las películas recomendadas, incluyendo el título, el año de estreno y la duración.*

In [8]:
def normalize_text(text):
    # Convertimos el texto a minúsculas y removemos acentos
        if isinstance(text, str):
            text = unidecode(text.lower())
        else:
            text = text.apply(lambda x: unidecode(x.lower()))
        return text

In [9]:
def recommend_movies(title):
    # Normalizamos el título sin diferenciar acentos
    title = normalize_text(title)
    
    # Obtenemos la película de entrada por título
    movie = df_movies[normalize_text(df_movies['title'].apply(normalize_text)) == title]
    
    if len(movie) == 0:
        return "No se encontró la película en la base de datos."
    
    # Obtenemos el índice de la película de entrada
    index = movie.index[0]
    
    # Obtenemos el resumen y género de la película de entrada
    overview = df_movies.loc[index, 'overview']
    genre = df_movies.loc[index, 'genres_names']
    
    # Calculamos la similitud basada en el resumen
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_movies['overview'])
    overview_similarities = cosine_similarity(tfidf_matrix[index], tfidf_matrix).flatten()
    
    # Filtramos películas similares por género
    similar_movies_indices = overview_similarities.argsort()[::-1]
    similar_movies = df_movies.loc[similar_movies_indices]
    similar_movies = similar_movies[similar_movies['genres_names'].str.contains(genre)].head(6)
    
    # Eliminamos la película de entrada de las recomendaciones
    similar_movies = similar_movies[similar_movies.index != index]
    
    # Convertimos el tiempo a enteros
    similar_movies['runtime'] = similar_movies['runtime'].astype(int)
    
    return similar_movies[['title', 'release_year', 'runtime']]


In [11]:
title = input("Ingrese el título de la película: ")
recommendations = recommend_movies(title)

if isinstance(recommendations, str):
    print(recommendations)
else:
    print(recommendations)

                                    title  release_year  runtime
8988   Batman Beyond: Return of the Joker          2000       74
17457               The Dark Knight Rises          2012      165
23862                   Batman vs Dracula          2005       83
14917          Batman: Under the Red Hood          2010       75
30101    Batman Unlimited: Monster Mayhem          2015       81
